### 先清洗一波我的训练数据

#### 0102 我发现有一些训练数据过了gpt4之后就做错了，gsm8k本身有算式和cot，我用规则造

#### 我发现规则也有一些漏洞，不要规则了，还是GPT4，但是我修正了一些问题，之前的instruction有问题

In [1]:
import os, re, json

In [2]:
with open("/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/train.jsonl", "r") as f:
    train_gsm8k = f.readlines()

In [ ]:
def build_my_format_data(one_line_input_data):
    question, answer = one_line_input_data["question"], one_line_input_data["answer"]
    analysis, answer = answer.split("\n####")
    analysis = analysis.strip("\n")
    answer = answer.strip("\n")
    step_analysis = analysis.split("\n")

In [4]:
json.loads(train_gsm8k[1])

{'question': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?',
 'answer': 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10'}

#### 0102之前的造数据方式

In [2]:
### 27号我有一波新的训练数据，看看成色。
import os, json
import re

In [3]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/hcot_gsm8k_gpt4_again_0124.json",
    "r",
) as f:
    hcot_gsm8k_new = f.readlines()

In [42]:
def fix_incomplete_tags_v2(text: str) -> str:
    tags = ["thought", "equation"]

    for tag in tags:
        open_tag = f"<{tag}>"
        close_tag = f"</{tag}>"
        incomplete_situation_open_tag_left = f"<{tag}"
        incomplete_situation_open_tag_right = f"{tag}>"
        incomplete_situation_close_tag_left = f"/{tag}>"
        incomplete_situation_close_tag_right = f"</{tag}"
        text_tmp = text.replace(open_tag, "&*#$%")
        text_tmp = text_tmp.replace(close_tag, "@#!$%")
        text_tmp = text_tmp.replace(incomplete_situation_close_tag_left, close_tag)
        text_tmp = text_tmp.replace(close_tag, "@#!$%")

        text_tmp = text_tmp.replace(incomplete_situation_close_tag_right, close_tag)
        text_tmp = text_tmp.replace(close_tag, "@#!$%")

        text_tmp = text_tmp.replace(incomplete_situation_open_tag_left, open_tag)
        text_tmp = text_tmp.replace(open_tag, "&*#$%")
        text_tmp = text_tmp.replace(incomplete_situation_open_tag_right, open_tag)
        text_tmp = text_tmp.replace(open_tag, "&*#$%")

        text_tmp = text_tmp.replace("&*#$%", open_tag)
        text_tmp = text_tmp.replace("@#!$%", close_tag)
        text = text_tmp
    return text


# Example usage
example_string = "This is a test equation>xample without proper tag. Here is another \n/thought> with no end."
fixed_string = fix_incomplete_tags_v2(example_string)
fixed_string

'This is a test <equation>xample without proper tag. Here is another \n</thought> with no end.'

In [27]:
json.loads(hcot_gsm8k_new[0])

{'id': '573a8111-5b87-4093-b009-e46f80b9281f',
 '_id': 1047832,
 'created_at': '2024-01-02 15:51:38',
 'updated_at': '2024-01-02 15:52:54',
 'status': 3,
 'response': '<|im_start|>[assistant](#inner_monologue)\\nDeciding on language: `English` detected from the user message. Using the same language for my reply and my suggested user responses.\\nDeciding on next action: `math_action` I will understand the question and analysis and return markdown format of my detailed thoughts and answer to this question.\\n[assistant](#math_action)\\nThe problem involves finding out how many clips Natalia sold in total over two months, given that she sold a certain number in one month (April) and half as many in another month (May).\\n<thought>Firstly, we know that Natalia sold 48 clips to her friends in April. In May, she sold half as many clips as she did in April. Therefore, we can find out how many clips she sold in May by dividing the number of clips she sold in April by 2.</thought>\\n<equation>

In [43]:
new_hcot_gsm8k_new = []
count_fix = []
for line in hcot_gsm8k_new:
    line_js = json.loads(line)
    one_response = line_js["response"]
    one_new_response = one_response.replace("\n\n", "\n")
    line_js["response"] = one_new_response
    if line_js["response"]:
        response_origin = line_js["response"]
        response_fix = fix_incomplete_tags_v2(response_origin)
        if response_fix != response_origin:
            count_fix.append((response_fix, response_origin))
        line_js["response"] = response_fix
        new_hcot_gsm8k_new.append(line_js)

In [44]:
with open("./data/GSM8K/hcot_gsm8k_fix_0126.jsonl", "w") as f:
    for line in new_hcot_gsm8k_new:
        f.write(json.dumps(line, ensure_ascii=False)+"\n")

### 构建带cot的直接训练数据

In [46]:
import os, json
from tqdm import tqdm

with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/hcot_gsm8k_fix_0126.jsonl",
    "r",
) as f:
    h_cot_gsm8k = f.readlines()


def build_direct_conversations(one_gsm8k_input):
    one_gsm8k_input["response"] = one_gsm8k_input["response"].replace(
        "[assistant'](#math_action)", "[assistant](#math_action)"
    )
    model_response = (
        one_gsm8k_input["response"]
        .split("[assistant](#math_action)")[1]
        .strip("\\n")
        .replace("\\n", "\n")
    )
    one_gsm8k_input["conversations"] = [
        {"from": "human", "value": one_gsm8k_input["meta"]["question"]},
        {"from": "gpt", "value": model_response},
    ]
    return one_gsm8k_input


h_cot_gsm8k_direct = []
for one_gsm8k in h_cot_gsm8k:
    one_gsm8k_js = json.loads(one_gsm8k)
    if one_gsm8k_js["response"]:
        h_cot_gsm8k_direct.append(build_direct_conversations(one_gsm8k_js))

In [48]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/h_cot_direct_conv_0126.json",
    "w",
) as f:
    json.dump(h_cot_gsm8k_direct, f, ensure_ascii=False)

In [49]:
len(h_cot_gsm8k_direct)

7473

In [19]:
h_cot_gsm8k_direct[0]

{'id': '4e47eef7-d7dc-427a-b1ed-6d2a511e38cf',
 '_id': 1061849,
 'created_at': '2024-01-24 15:00:25',
 'updated_at': '2024-01-24 15:02:31',
 'status': 3,
 'channel': 'default',
 'response': "[assistant](#inner_monologue)\\nDeciding on language: `English` detected from the user message. Using the same language for my reply and my suggested user responses.\\nDeciding on next action: `math_action` I will understand the question and analysis and return markdown format of my detailed thoughts and answer to this question.\\n[assistant](#math_action)\\nThe problem involves finding out how much credit Mary still needs to pay before her next shopping trip. We know that she spent her full credit limit of $100, but has only paid back a portion of it so far.\n\n<thought>Firstly, we need to calculate how much Mary has already paid back by adding up all her payments.</thought>\n<equation>Total amount paid = $15 + $23 = $38</equation>\n\n<thought>Next, we subtract what she's already paid from her tot

#### 构建hcot的cot model的训练数据

In [50]:
import re


def remove_thoughts(text):
    """
    Removes all occurrences of <thought>xxx</thought> from the text.

    Args:
    text (str): The input string containing <thought>xxx</thought> tags.

    Returns:
    str: The modified string with <thought>xxx</thought> tags removed.
    """
    # Pattern to match <thought>xxx</thought> and optional preceding and following newlines
    pattern = r"(?<=\n)<thought>.*?</thought>(?=\n)"

    # Replace all occurrences with an empty string
    modified_text = re.sub(pattern, "<[COT]>", text, flags=re.DOTALL)

    return modified_text


def extract_thoughts_accumulative(problem, text):
    pairs = []
    previous_end = 0
    # prefix = f"Solve the following math word problem with your step-by-step analysis, {problem}. I will provide the previous analysis and equations, your job is to show me your thought of the next action, you do not need to give the calculation equation.\n\n"
    prefix = f"{problem}"

    while "<thought>" in text[previous_end:]:
        # Find the next <thought> tag
        start_thought = text.find("<thought>", previous_end)
        end_thought = text.find("</thought>", start_thought)

        # Break the loop if no more <thought> tag is found
        if start_thought == -1 or end_thought == -1:
            break

        # Extract the input and output
        input_text = text[:start_thought].strip()
        output_text = text[
            start_thought + 9 : end_thought
        ].strip()  # 9 is the length of '<thought>'

        # Append the pair to the result
        input_text_tmp = prefix + remove_thoughts(input_text)

        pairs.append(
            {
                "conversations": [
                    {"from": "human", "value": input_text_tmp + "\n<[COT]>"},
                    {"from": "gpt", "value": output_text},
                ],
                "Query": input_text_tmp,
                "COT_target": "\n<[COT]>",
                "thought_target": output_text,
            }
        )

        # Update the end position for the next iteration
        previous_end = end_thought + 10

    return pairs

In [51]:
cot_model_train_data = []
for one_gsm8k_direct in h_cot_gsm8k_direct:
    new_train_data = extract_thoughts_accumulative(
        one_gsm8k_direct["conversations"][0]["value"],
        one_gsm8k_direct["conversations"][1]["value"],
    )
    cot_model_train_data.extend(new_train_data)

In [22]:
cot_model_train_data = []
for one_gsm8k_direct in h_cot_gsm8k_direct:
    new_train_data = extract_thoughts_accumulative(
        one_gsm8k_direct["conversations"][0]["value"],
        one_gsm8k_direct["conversations"][1]["value"],
    )
    cot_model_train_data.extend(new_train_data)

In [52]:
h_cot_gsm8k_direct[0]["conversations"][1]["value"]

"The problem involves finding out how much credit Mary still needs to pay before her next shopping trip. We know that she spent her full credit limit of $100, but has only paid back a portion of it so far.\n<thought>Firstly, we need to calculate how much Mary has already paid back by adding up all her payments.</thought>\n<equation>Total amount paid = $15 + $23 = $38</equation>\n<thought>Next, we subtract what she's already paid from her total credit limit to find out how much she still owes.</thought>\n<equation>Credit remaining = Total Credit - Amount Paid = $100 - $38 = $62</equation>\nSo, Mary needs to pay an additional $62 before her next shopping trip."

In [53]:
# 检查一下构造的数据
import random
random_index = random.sample(list(range(7000)), k=20)

In [55]:
cot_model_train_data[random_index[0]]

{'conversations': [{'from': 'human',
   'value': "Joey wants to buy the latest released pair of designer High Jump basketball sneakers. He plans to mow 3 neighbors’ lawns for $8 a lawn, sell 2 collectible figures to his friends for $9 each, and work an after-school job for 10 hours at $5 per hour. If his earnings just cover the price of the High Jump sneakers, how much do the shoes cost?The problem involves calculating Joey's total earnings from three different sources, which are mowing lawns, selling collectible figures, and working an after-school job. The sum of these earnings is equal to the cost of the High Jump sneakers he wants to buy.\n<[COT]>\n<equation>Earnings from mowing = 3 * $8 = $24</equation>\n<[COT]>"},
  {'from': 'gpt',
   'value': 'Next, we calculate his earnings from selling 2 collectible figures at $9 each.'}],
 'Query': "Joey wants to buy the latest released pair of designer High Jump basketball sneakers. He plans to mow 3 neighbors’ lawns for $8 a lawn, sell 2 co

In [54]:
jesus = extract_thoughts_accumulative(
    h_cot_gsm8k_direct[0]["conversations"][0]["value"],
    h_cot_gsm8k_direct[0]["conversations"][1]["value"],
)

In [71]:
for d in cot_model_train_data:
    d["conversations"][1]["value"] = d["conversations"][1]["value"] + "</[COT]>"

In [73]:
cot_model_train_data[1]

{'conversations': [{'from': 'human',
   'value': 'Mary does her grocery shopping on Saturday. She does her shopping only at a specific store where she is allowed a credit of $100, which must be paid in full before her next shopping trip. That week she spent the full credit limit and paid $15 of it on Tuesday and $23 of it on Thursday. How much credit will Mary need to pay before her next shopping trip?The problem involves finding out how much credit Mary still needs to pay before her next shopping trip. We know that she spent her full credit limit of $100, but has only paid back a portion of it so far.\n<[COT]>\n<equation>Total amount paid = $15 + $23 = $38</equation>\n<[COT]>'},
  {'from': 'gpt',
   'value': "Next, we subtract what she's already paid from her total credit limit to find out how much she still owes.</[COT]>"}],
 'Query': 'Mary does her grocery shopping on Saturday. She does her shopping only at a specific store where she is allowed a credit of $100, which must be paid i

In [74]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/COT_model_train_data_kun_idea_0126.json",
    "w",
) as f:
    json.dump(cot_model_train_data, f, ensure_ascii=False)

#### 构建hcot-model-llm部分的训练数据

In [57]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/h_cot_direct_conv_0126.json",
    "r",
) as f:
    h_cot_gsm8k_direct = json.load(f)

In [58]:
import re


def remove_thoughts(text):
    """
    Removes all occurrences of <thought>xxx</thought> from the text.

    Args:
    text (str): The input string containing <thought>xxx</thought> tags.

    Returns:
    str: The modified string with <thought>xxx</thought> tags removed.
    """
    # Pattern to match <thought>xxx</thought> and optional preceding and following newlines
    pattern = r"(?<=\n)<thought>.*?</thought>(?=\n)"

    # Replace all occurrences with an empty string
    modified_text = re.sub(pattern, "<[COT]>", text, flags=re.DOTALL)

    return modified_text

In [60]:
one_text = "The problem involves finding out how much money Mr. Sam shared between his two sons, given that Ken received a certain amount, and Tony received twice as much as Ken.\n<thought>Firstly, since Tony got twice as much money as Ken did, we can find out how much Tony got by multiplying Ken's share by 2.</thought>\n<equation>Tony's share = 2 * $1750 = $3500</equation>\n<thought>Next, to find out the total amount of money shared by Mr. Sam between his two sons, we add up what each son received.</thought>\n<equation>Total amount shared = $1750 (Ken's share) + $3500 (Tony's share) = $5250</equation>\nSo, Mr. Sam shared a total of $5250 between his two sons."
remove_thoughts(one_text)

"The problem involves finding out how much money Mr. Sam shared between his two sons, given that Ken received a certain amount, and Tony received twice as much as Ken.\n<[COT]>\n<equation>Tony's share = 2 * $1750 = $3500</equation>\n<[COT]>\n<equation>Total amount shared = $1750 (Ken's share) + $3500 (Tony's share) = $5250</equation>\nSo, Mr. Sam shared a total of $5250 between his two sons."

In [61]:
h_cot_gsm8k_hcot_llm_model = []
for data in h_cot_gsm8k_direct:
    conv = data["conversations"]
    conv_response = conv[1]["value"]
    conv_response_with_cot = remove_thoughts(conv_response)
    conv[1]["value"] = conv_response_with_cot
    h_cot_gsm8k_hcot_llm_model.append(data)

In [63]:
with open(
    "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/hcot_llm_data_0126.json",
    "w",
) as f:
    json.dump(h_cot_gsm8k_hcot_llm_model, f, ensure_ascii=False)

In [68]:
h_cot_gsm8k_hcot_llm_model[0]

{'id': '4e47eef7-d7dc-427a-b1ed-6d2a511e38cf',
 '_id': 1061849,
 'created_at': '2024-01-24 15:00:25',
 'updated_at': '2024-01-24 15:02:31',
 'status': 3,
 'channel': 'default',
 'response': "[assistant](#inner_monologue)\\nDeciding on language: `English` detected from the user message. Using the same language for my reply and my suggested user responses.\\nDeciding on next action: `math_action` I will understand the question and analysis and return markdown format of my detailed thoughts and answer to this question.\\n[assistant](#math_action)\\nThe problem involves finding out how much credit Mary still needs to pay before her next shopping trip. We know that she spent her full credit limit of $100, but has only paid back a portion of it so far.\n<thought>Firstly, we need to calculate how much Mary has already paid back by adding up all her payments.</thought>\n<equation>Total amount paid = $15 + $23 = $38</equation>\n<thought>Next, we subtract what she's already paid from her total c

In [ ]:
import os,json

In [ ]:
with open("/mnt/pfs/zitao_team/tianqiaoliu/Project/papers/hidden_cot/data/GSM8K/hcot_llm_data.json", "r") as f:
    file = json.load(f)

In [ ]:
has_thought = []
for one in file:
    one_content = one["conversations"][1]["value"]
    if "<thought>" in one_content:
        has_thought.append(one)

In [ ]:
len(has_thought)

248

In [ ]:
has_thought[0]

{'question': 'USER: Question: A concert ticket costs $40. Mr. Benson bought 12 tickets and received a 5% discount for every ticket bought that exceeds 10. How much did Mr. Benson pay in all?\n Analysis: Mr. Benson had a 5% discount for each of the 12 - 10 = <<12-10=2>>2 tickets.\nSo, those two tickets had a $40 x 5/100 = $<<40*5/100=2>>2 discount each.\nHence, each ticket cost $40 - $2 = $<<40-2=38>>38 each.\nThus, two discounted tickets amount to $38 x 2 = $<<38*2=76>>76.\nAnd the other ten tickets amount to $40 x 10 = $<<40*10=400>>400.\nHence, Mr. Benson paid a total of $400 + $76 = $<<400+76=476>>476.\n Answer:476',
 'raw_result': {'id': 'chatcmpl-8SRB5JiwuLiCvgGboHoupKbLk8ywt',
  'object': 'chat.completion',
  'created': 1701788347,
  'model': 'gpt-4',
  'choices': [{'index': 0,
    'message': {'role': 'assistant',
     'content': '<|im_start|>[assistant](#inner_monologue)\\nDeciding on language: `English` detected from the user message. Using the same language for my reply and my

#### 查看一下测试数据

In [2]:
with open("/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/data/GSM8K/reformat_test.json", "r") as f:
    reformat_gsm8k_test = json.load(f)

In [3]:
reformat_gsm8k_test[0]

{'analysis': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.',
 'answer': ' 18',
 'instruction': '',
 'input': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"}